In [ ]:
## My Redshift query editor is not working via console, so I decided to query the tables using boto3

In [ ]:
import boto3
import psycopg2
import configparser
import pandas as pd

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

CLUSTER_ENDPOINT = config.get("CLUSTER", "HOST")
PORT = config.get("CLUSTER", "DWH_PORT")

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME", "CLUSTER_ENDPOINT"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME, CLUSTER_ENDPOINT]
             })


In [ ]:
# Connect to Redshift cluster
try:
    conn = psycopg2.connect(
        host=CLUSTER_ENDPOINT,
        port=PORT,
        database=DWH_DB,
        user=DWH_DB_USER,
        password=DWH_DB_PASSWORD
    )
except Exception as e:
    print("Error connecting to Redshift cluster:", e)
    raise e

In [ ]:
# List all tables
cur = conn.cursor()
cur.execute("SELECT tablename FROM pg_tables WHERE schemaname='public'")
tables = cur.fetchall()

# Print the list of tables
print("Existing tables:")
for table in tables:
    print(table[0])

In [ ]:
# Fetch and count rows for each table
for table in tables:
    table_name = table[0]
    cur.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cur.fetchone()[0]
    print(f"Table {table_name} has {count} rows")


In [ ]:
# Close connection
conn.close()